In [2]:
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D, MaxPooling2D)
from keras.layers.normalization import (BatchNormalization)
from keras.models import Sequential, load_model
from keras import regularizers
from keras.utils import to_categorical
from PIL import Image
from models import models

import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import glob
from PIL import Image
import time
import warnings



In [3]:
# model
def c3d(num_classes, shape):
    
    
    model = Sequential()
    # conv_1
    conv_1 = Conv3D(32, (7,5,5), input_shape=shape, strides=(1, 2, 2))
    model.add(conv_1)
    print(conv_1.output_shape)
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
    
    # conv_2 
    conv_2 = Conv3D(64, (3,3,3))
    model.add(conv_2)
    print(conv_2.output_shape)
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
    
    # conv_3
    conv_3 = Conv3D(128, (2,2,2))
    model.add(conv_3)
    print(conv_3.output_shape)
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))


    #
    model.add(Flatten())
    d1 = Dense(256)
    model.add(d1)
    print (d1.output_shape)
    model.add(Dropout(0.2))
    
    d2 = Dense(256)
    model.add(d2)
    print (d2.output_shape)
    model.add(Dropout(0.2))
    
    d3 = Dense(num_classes, activation='softmax')
    model.add(d3)
    print (d3.output_shape)
    return model

In [4]:
# process input data
def dataPreProcess(x_train_ind, num_videos, frame_dir, shape):
    tic = time.time()
    
    # create input data matrix
    if len(shape) == 4:
        num_frames, h, w, c = shape
        data = np.zeros( (num_videos, num_frames, h, w, c))
    else:
        h, w, c = shape
        data = np.zeros( (num_videos, h, w, c) )
    
    # read videos frames
    count_video = 0    
    for video_ind in x_train_ind:
        # check video frames available
        path = frame_dir +'/video'+str(video_ind)
        if not os.path.exists(path): continue
            
        # read frames
        count_video += 1
        for fi in range(1,num_frames+1):
            frame = Image.open(path+'/frame'+str(fi)+'.jpg')
            frame = frame.resize( (h,w), Image.ANTIALIAS)
            frame = np.asarray( frame, dtype="int32" ) # transform to array
            frame.flags.writeable = True
            frame = frame.astype(np.float64)
            # im_arr = preprocess_input(im_arr)   substraction, normaliztion ....
            data[count_video-1, fi-1] = frame
        
        if count_video%250 == 0:
            toc = time.time()-tic
            print ("finished %d video frames from %d videos, elapsed time:%d s" %(count_video, video_ind, toc) ) 
        
        if count_video == num_videos:
            print ("finished all processing")
            break
        
    return data


In [23]:
# parameters setting
num_frames = 10
h, w, c = 64, 64, 3
shape = (num_frames, h, w, c)
num_videos = 50
curr_path = os.getcwd()
frame_dir = curr_path + '/datasets/frames'

x_train_ind = np.load(curr_path+'/datasets/train_ind_above400.npy')
x_test_ind = np.load(curr_path+'/datasets/test_ind_above400.npy')


y_train = np.load(curr_path+'/datasets/y_train_mapped_above400.npy')
y_test = np.load(curr_path+'/datasets/y_test_mapped_above400.npy')



if num_videos > len(x_train_ind):
    warnings.warn("Exceed training data szie", DeprecationWarning)
    num_videos = len(x_train_ind)
y_train = y_train[:num_videos]
    
    
x_train = dataPreProcess(x_train_ind, num_videos, frame_dir, shape )
y_train = to_categorical(y_train, num_classes=num_classes)

# x_test = ataPreProcess(x_test_ind, num_videos, frame_dir, shape )
# y_test = to_categorical(y_test, num_classes=num_classes)



finished all processing


In [24]:
print( len(x_train), len(y_train),  num_classes)
print( x_train.shape, y_train.shape)

50 51 10
(50, 10, 64, 64, 3) (51, 10)


In [7]:
# train
model = models(num_classes, "c3d", num_frames = 10, reg = 1e-2)
history = model.model.fit(x, y, epochs=30, batch_size=32, validation_split = 0.2, verbose = 1)


# model = c3d(num_classes, (num_frames, h, w, c))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# history = model.fit(x, y, epochs=30, batch_size=32, validation_split = 0.2, verbose = 1)


(10, 64, 64, 3)
(None, 4, 30, 30, 32)
(None, 2, 13, 13, 64)
(None, 1, 5, 5, 128)
(None, 256)
(None, 256)
(None, 20)
Train on 4000 samples, validate on 1000 samples
Epoch 1/30
4000/4000 [==============================] - 9s - loss: 5.8774 - acc: 0.1110 - val_loss: 5.5244 - val_acc: 0.0900
Epoch 2/30
4000/4000 [==============================] - 8s - loss: 4.6458 - acc: 0.1750 - val_loss: 4.4671 - val_acc: 0.1310
Epoch 3/30
4000/4000 [==============================] - 8s - loss: 3.9633 - acc: 0.1950 - val_loss: 3.9126 - val_acc: 0.1720
Epoch 4/30
4000/4000 [==============================] - 8s - loss: 3.4728 - acc: 0.2132 - val_loss: 3.5409 - val_acc: 0.1730
Epoch 5/30
4000/4000 [==============================] - 8s - loss: 3.1336 - acc: 0.2430 - val_loss: 3.3759 - val_acc: 0.1750
Epoch 6/30
4000/4000 [==============================] - 8s - loss: 2.8750 - acc: 0.2593 - val_loss: 3.1076 - val_acc: 0.1790
Epoch 7/30
4000/4000 [==============================] - 9s - loss: 2.6733 - acc: 0.294

KeyboardInterrupt: 

In [23]:
# plot training history
# print(history.history.keys())
# plt.subplots(121)
plt.plot(history.history['acc'])
# # plt.plot(history.history['val_acc'])

TclError: no display name and no $DISPLAY environment variable